<a href="https://colab.research.google.com/github/LeonMilosevic/e-bay_phone_evaluator/blob/main/phone_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## E-bay Phone Evaluator

### Steps to take:

- ##### We will scrape data from ebay with a custom ebay scraper class found here: https://github.com/LeonMilosevic/ebay-phone-scraper
- ##### We will create functions to help us clean our data
- ##### We will perform EDA on our data
- ##### We will create a baseline model
- ##### Do Feature Engineering and Hyperparametars Tuning to improve the performance of our model
- ##### We will export the model and use it for our Flask API to enable the app for usage, <br /> you can find it here: https://github.com/LeonMilosevic/ebay_phone_price_predictor_api

## Data Scraping

#### Install the package from github

In [ ]:
!pip install git+https://github.com/LeonMilosevic/ebay-phone-scraper

#### Installing necessery packages to use ebay-phone-scraper

In [ ]:
!pip install fake-useragent

In [ ]:
!pip install pytest ipython_pytest

#### Testing the package

In [4]:
import ipython_pytest
%load_ext ipython_pytest

In [5]:
%%pytest
from scraper import scraper
from fake_useragent import UserAgent
import pandas as pd
import pytest

def test_check_brandIdApple():
  result = scraper.check_brandId('Apple')

  assert result == 319682

def test_check_brandIdLG():
  result = scraper.check_brandId('LG')

  assert result == 353985

def test_check_brandIdHuawei():
  result = scraper.check_brandId('Huawei')

  assert result == 349965

def test_check_brandIdSamsung():
  result = scraper.check_brandId('Samsung')

  assert result == 352130

def test_check_brandIdError():
  with pytest.raises(ValueError):
      scraper.check_brandId('bad')

def test_check_conditionNew():
  result = scraper.check_condition("new")
  print(result)
  assert result == 1000

def test_check_conditionUsed():
  result = scraper.check_condition('used')

  assert result == 3000

def test_check_conditionError():
    with pytest.raises(ValueError):
        scraper.check_condition('something else')

def test_calculate_number_of_pages():
    result = scraper.calculate_number_of_pages(48)

    assert result == 1

def test_create_datafrmae():
  brand = "Apple"
  phone_price = ['123', '456', '789']
  phone_model = ['galaxy', 'iphone 7', 'iphone 8']
  phone_ram = ['3 gb', '4gb', '5gb']
  phone_storage = ['64gb', '128gb', '16gb']
  phone_processor = ['some processor', 'some processor2', 'some processor3']
  phone_camera = ['16', '8', '12']
  condition = 0

  result = scraper.create_dataframe(
    brand=brand, 
    condition=condition,
    phone_price=phone_price, 
    phone_model=phone_model, 
    phone_ram=phone_ram, 
    phone_storage=phone_storage, 
    phone_processor=phone_processor, 
    phone_camera=phone_camera
    )
  assert type(result) == pd.DataFrame

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.10.0, pluggy-0.7.1
rootdir: /tmp/tmp5kkvrjyl, inifile:
plugins: typeguard-2.7.1
collected 10 items

_ipytesttmp.py ..........                                                [100%]

========================== 10 passed in 0.32 seconds ===========================


#### Imports to be used

In [1]:
import pandas as pd
import numpy as np

#### Data Cleaning functions

In [40]:
# defining functions to be used
def combine_data(list_of_dataframes: list) -> pd.DataFrame:
  """ 
  Joins passed list of dataframes
  
  Arg:
    list_of_dataframes (list)

  returns:
    pd.DataFrame object of passed dataframes
  """
  return pd.concat(list_of_dataframes)

def sum_cameras(df: pd.DataFrame) -> list:
  ''' takes a dataframe, finds cameras with higher str value than 6, sums them and returnes the summed values
      
      args:
            df (pd.DataFrame): dataframe to be operated on

      returns:
            summed_cameras (list): list of summed values passed from the dataframe
  '''
  summed_cameras = []  

  for item in df[df['camera'].str.len() > 6]['camera']:
    result = [x.strip() for x in item.split(',')]
    for i in range(len(result)):
      if result[i] == '8.0.0':
        result[i] = float(8)
      else:
        result[i] = float(result[i])
    summed_cameras.append(sum(result))
  
  return summed_cameras

def rename_phone_model(bad_name_model: str, good_name_model: str, df: pd.DataFrame) -> None:
  """ 
  Renames a given name of the model, with a provided name of the model
  
  Arg:
    bad_name_model (str) : name to be replaced
    good_name_model (str) : name to be replaced with
    df (pd.DataFrame) : dataframe to be cleaned

  returns:
        None
  """
  df[df['model'] == bad_name_model] = good_name_model

def remove_unwanted_models(df: pd.DataFrame, model_name: str) -> None:
  """ 
  Removes given models

  Arg:
    bad_name_model (str) : name to be replaced
    good_name_model (str) : name to be replaced with
    df (pd.DataFrame) : dataframe to be cleaned

  returns:
        None
  """
  models = df[df['model'] == model_name].index

  df.drop(models, inplace=True)

def clean_columns(df: pd.DataFrame, characters: list, column: str, replace_char: str = "") -> pd.DataFrame:
  '''
  Accepts a pd.DataFrame, characters, column. Removes passed characters, returns clean pd.DataFrame

  Arg: df (pd.DataFrame): DataFrame to be cleaned,
       characters (list): list of characters to be removed
       column (str): name of the column to be cleaned
       replace_char (str) : default value is empty, but if specified, can replace the value as well
  returns:
      df (pd.DataFrame): cleaned DataFrame
  '''
  for char in characters:
    df[column] = df[column].str.replace(char, replace_char)

  return df 

def convert_data_to_numeric(df: pd.DataFrame) -> None:
  '''
  Accepts a pd.DataFrame, converts its columns to numeric

  Arg: df (pd.DataFrame): DataFrame to be converted to numeric
  
  returns:
      df (pd.DataFrame): numeric DataFrame
  '''

  df['storage'] = pd.to_numeric(df['storage'])
  df['price'] = pd.to_numeric(df['price'])
  df['camera'] = pd.to_numeric(df['camera'])
  df['ram'] = pd.to_numeric(df['ram'])

def remove_rows_with_false_values(df: pd.DataFrame) -> None:
  '''
  Accepts a pd.DataFrame, removes rows that have same value in each column

  Arg: df (pd.DataFrame): DataFrame to be cleaned
  
  returns:
        None
  '''

  df.drop(df[df['processor'] == df['camera']].index, inplace=True)

def populate_columns_by_model_mean(df: pd.DataFrame) -> None:
  '''
  Accepts a pd.DataFrame, replaces ram memory in each row by model mean value

  Arg: df (pd.DataFrame): DataFrame to be cleaned
  
  returns:
        None
  '''
  models_ram_values = df.groupby('model').agg({'ram': 'mean'})['ram']
  models_camera_values = df.groupby('model').agg({'camera': 'mean'})['camera']

  for model, value in models_ram_values.items():
    temp_index = df[df['model'] == model].index

    for i in temp_index:
      df.at[i, 'ram'] = value
  
  for model, value in models_camera_values.items():
    temp_index = df[df['model'] == model].index

    for i in temp_index:
      df.at[i, 'camera'] = value

def replace_mb_to_gb(df: pd.DataFrame, mb_limit: int, column_name: str) -> None:
  '''
  Accepts a pd.DataFrame, limit to be passed of the mb, name of the column,
  moves decimal by 3 places to convert it from GB to MB

  Arg: 
    df (pd.DataFrame): DataFrame to be cleaned
    mb_limit (int) : limit value that we want for specific column to be searched for
    column_name (str) : name of the column to be transformed from MB to GB
  
  returns:
        None
  '''
  mb = df[df[column_name] > mb_limit][column_name].tolist()
  mb_index = df[df[column_name] > mb_limit].index
  
  converted = [(ram / 1000) for ram in mb]

  for i in range(len(mb_index)):
    df.at[mb_index[i], column_name] = converted[i]

def replace_specific_value(df: pd.DataFrame, original_value: str, new_value: str, column_name) -> None:
  '''
  Accepts a pd.DataFrame, replaces old value with a new value in the specified column

  Arg: 
    df (pd.DataFrame): DataFrame to be changed
    original_value (int) : value we want to replace
    new_value (str, int) : value that will change original_value
    column_name (str) : name of the column that needs to be changed
  
  returns:
        None
  '''
  original_value_idx = df[df[column_name] == original_value].index

  for i in range(len(original_value_idx)):
    df.at[original_value_idx[i], column_name] = new_value

def multi_camera_phones_to_single_value(df: pd.DataFrame) -> None:
  '''
  accepts a dataframe, sums cameras from camera column and replaces their value with summed value

  Arg: 
    df (pd.DataFrame): DataFrame to be changed
  
  returns:
        None
  '''
  cameras_to_be_replaced = df[df['camera'].str.len() > 6].index
  summed_cameras = sum_cameras(df)

  for i in range(len(cameras_to_be_replaced)):
    df.at[cameras_to_be_replaced[i], 'camera'] = summed_cameras[i]

# onehot encode function
def onehot_encode(df, column, prefix):
  '''
  Hot encodes a given column, and adds it to a Dataframe

  Arg: 
    df (pd.DataFrame): DataFrame to be hot encoded
    prefix (str) : prefix to be given to hot encoded columns
    column (str) : name of the column that needs to be changed
  
  returns:
        None
  '''
  df = df.copy()
  dummies = pd.get_dummies(df[column], prefix=prefix)
  df = pd.concat([df, dummies], axis=1)
  df = df.drop(column, axis=1)
  return df

def update_specific_model_ram(model_name: str, ram: int, df: pd.DataFrame) -> None:
  '''
  Accepts a pd.DataFrame, replaces ram memory in for a given model name

  Arg: 
    df (pd.DataFrame): DataFrame to be cleaned
    model_name (str) : name of the model
    ram (int) : value to be updated
  
  returns:
        None
  '''
  model_index = df[df['model'] == model_name].index

  for i in model_index:
    df.at[i, 'ram'] = ram

#### Data Wrangling

In [3]:
apple_new_data = pd.read_csv('Apple_new_data.csv')
apple_used_data = pd.read_csv('Apple_used_data.csv')
huawei_new_data = pd.read_csv('Huawei_new_data.csv')
huawei_used_data = pd.read_csv('Huawei_used_data.csv')
samsung_new_data = pd.read_csv('Samsung_new_data.csv')
samsung_used_data = pd.read_csv('Samsung_used_data.csv')

In [4]:
# adding condition to data
apple_new_data['condition'] = 'new'
apple_used_data['condition'] = 'used'

huawei_new_data['condition'] = 'new'
huawei_used_data['condition'] = 'used'

samsung_new_data['condition'] = 'new'
samsung_used_data['condition'] = 'used'

In [5]:
# join dataframes
list_of_df = [apple_new_data,
                 apple_used_data,
                 huawei_new_data,
                 huawei_used_data,
                 samsung_new_data,
                 samsung_used_data]

phone_data = combine_data(list_of_df)

In [6]:
phone_data.shape

(14853, 8)

In [7]:
phone_data.isnull().sum()

price        6600
model        8517
ram          8024
storage      8492
processor    8612
camera       8901
brand           0
condition       0
dtype: int64

- Replacable Data: RAM, Processor and Camera resolution.



- Irreplaceable Data: price, model and storage, because we don't know precise sold price of the phone or correct model, as well same models come with different storage regardless of other specifics.


In [8]:
# dropping rows where price, model and storage is NaN
phone_data = phone_data.dropna(subset=['price', 'model', 'storage'])

The data contains extra characters in columns, we will remove all the string characters and leave only numeric characters

In [9]:
phone_data = clean_columns(phone_data, ['$', ','], 'price')
phone_data = clean_columns(phone_data, ['GB', 'gb', 'mb', 'MB'], 'storage')
phone_data = clean_columns(phone_data, ['MP', 'mb'], 'camera')
phone_data = clean_columns(phone_data, ['MB', 'GB', 'gb'], 'ram')
phone_data = clean_columns(phone_data, ['+'], 'camera', replace_char=",")

Some rows as well contain false values, <br />
example: all columns of a row are represented by a value of 1, we will remove those

In [10]:
remove_rows_with_false_values(phone_data)

In [11]:
phone_data.reset_index(drop=True, inplace=True)

Some models have more than one camera, or camera is named as Vga so we will need to sum the values of them and replace Vga with a numeric value

In [12]:
multi_camera_phones_to_single_value(phone_data)
replace_specific_value(phone_data, 'Vga', 43, 'camera')

Convert the data to numeric values

In [13]:
convert_data_to_numeric(phone_data)

We will use a function to populate ram missing values

In [41]:
populate_columns_by_model_mean(phone_data)
update_specific_model_ram('Apple iPhone 12 Pro Max', 6, phone_data)
update_specific_model_ram('Apple iPhone 12 mini', 4, phone_data)

In [42]:
phone_data.isnull().sum()

price         0
model         0
ram          54
storage       0
processor    97
camera       35
brand         0
condition     0
dtype: int64

In [72]:
phone_data.groupby('model')['processor'].value_counts()

model                          processor  
Apple iPhone (1st Generation)  Single Core      4
Apple iPhone 11                Hexa Core      202
Apple iPhone 11 Pro            Hexa Core       80
Apple iPhone 11 Pro Max        Hexa Core      115
Apple iPhone 12                Hexa Core      144
                                             ... 
Samsung Galaxy Z Fold2 5G      Octa Core       15
                               Dual Core        2
Xiaomi Mi 10T Lite             Octa Core        1
Y6                             Quad Core       10
iPhone 6s.                     Dual Core        4
Name: processor, Length: 240, dtype: int64

In [43]:
phone_data[phone_data.isnull().any(axis=1)]

,price,model,ram,storage,processor,camera,brand,condition
285,1641.34,Apple iPhone 12 Pro,6.0,128,NaN,12.0,Apple,new
1817,179.43,Apple iPhone 8,2.0,64,NaN,12.0,Apple,used
2159,61.17,5,NaN,16,NaN,NaN,Apple,used
2394,61.17,5,NaN,16,NaN,NaN,Apple,used
2801,679.64,Apple iPhone 8,2.0,256,NaN,12.0,Apple,used
...,...,...,...,...,...,...,...,...
5648,122.32,Samsung Galaxy Ace 2,768.0,4,NaN,5.0,Samsung,used
5651,15.63,Samsung Galaxy Ace,NaN,150,NaN,5.0,Samsung,used
5661,13.59,Samsung E1080,NaN,1,NaN,NaN,Samsung,used
5662,24.45,Samsung Galaxy Fame,512.0,4,NaN,5.0,Samsung,used
